In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy
import datetime
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

In [ ]:
%config Completer.use_jedi = False

In [ ]:
df=pd.read_csv('../input/original-train/train.csv')
pd.set_option('display.max_columns',None)

In [ ]:
df.head()

# 异常数处理

In [ ]:
# 数据类型分组
int_data=[]
str_data=[]

for i in range(len(df.iloc[0])):
    if isinstance(df.iloc[0,i],(int,float)):
        int_data.append(df.columns[i])
    else:
        str_data.append(df.columns[i])
        
print(int_data)
print(str_data)

In [ ]:
for data in [df]:
    data['issueDate'] = pd.to_datetime(data['issueDate'],format='%Y-%m-%d')
    startdate = datetime.datetime.strptime('2007-06-01', '%Y-%m-%d')
    data['issueDateDT'] = data['issueDate'].apply(lambda x: x-startdate).dt.days

In [ ]:
pivot = pd.pivot_table(df,index = ['grade'],columns = ['issueDateDT'],values = ['loanAmnt'],aggfunc = np.sum)

In [ ]:
def employmentLength_to_int(s):
    if pd.isnull(s):
        return s
    else:
        return np.int8(s.split()[0])
for data in [df]:
    data['employmentLength'].replace(to_replace='10+ years', value='10 years', inplace=True)
    data['employmentLength'].replace('< 1 year', '0 years', inplace=True)
    data['employmentLength'] = data['employmentLength'].apply(employmentLength_to_int)

In [ ]:
for data in [df]:
    data['earliesCreditLine'] = data['earliesCreditLine'].apply(lambda s: int(s[-4:]))

In [ ]:
# 区分连续性变量以及离散型变量
a_list=[3,5,8,9,16,17]

numerical_noserial_fea=[]
for i in a_list:
    numerical_noserial_fea.append(int_data[i])

    
numerical_serial_fea=[]

for i in int_data:
    if i not in numerical_noserial_fea:
        numerical_serial_fea.append(i)

In [ ]:
# 使用截尾对异常值进行清洗
def outliers_proc(data, col_name, scale=3):
    """
        用于截尾异常值， 默认用box_plot(scale=3)进行清洗
        param:
            data： 接收pandas数据格式
            col_name: pandas列名
            scale: 尺度
    """
    data_col = data[col_name]
    Q1 = data_col.quantile(0.25) # 0.25分位数
    Q3 = data_col.quantile(0.75)  # 0,75分位数
    IQR = Q3 - Q1
    
    data_col[data_col < Q1 - (scale * IQR)] = Q1 - (scale * IQR)
    data_col[data_col > Q3 + (scale * IQR)] = Q3 + (scale * IQR)

    return data[col_name]

In [ ]:
for i in numerical_serial_fea:
    df[i]=outliers_proc(df,i)

In [ ]:
df=df.drop(['n11','n12','n13','n0','pubRecBankruptcies','pubRec','delinquency_2years'],axis=1)

In [ ]:
for i in ['n11','n12','n13','n0','pubRecBankruptcies','pubRec','delinquency_2years']:
    numerical_serial_fea.remove(i)

In [ ]:
##grade 以及subgrade 的目标违约均值
for col in ['grade', 'subGrade']: 
    temp_dict = df.groupby([col])['isDefault'].agg(['mean']).reset_index().rename(columns={'mean': col + '_target_mean'})
    temp_dict.index = temp_dict[col].values
    temp_dict = temp_dict[col + '_target_mean'].to_dict()

    df[col + '_target_mean'] = df[col].map(temp_dict)

##利率/贷款总额
df.loc[:,'interestRate/loanAmnt'] = df.interestRate/df.loanAmnt

##年收入/贷款总额
df.loc[:,'annualIncome/loanAmnt'] = df.annualIncome/df.loanAmnt

##分期付款金额/贷款总额
df.loc[:,'installment/loanAmnt'] = df.installment/df.loanAmnt

##年收入/分期付款金额
df.loc[:,'annualIncome/installment'] = df.annualIncome/df.installment

##年收入/就业年限
df.loc[:,'annualIncome/employmentLength'] = df.annualIncome/df.employmentLength

##年收入*就业年限
df.loc[:,'annualIncome*employmentLength'] = df.annualIncome/df.employmentLength

##借款人信用档案中未结信用额度的数量/ 借款人信用档案中当前的信用额度总数
df.loc[:,'openAcc/totalAcc'] = df.openAcc/df.totalAcc

##借款人信用档案中未结信用额度的数量/ 借款人最早报告的信用额度开立月份
df.loc[:,'openAcc/earliesCreditLine'] = df.openAcc/df.earliesCreditLine

In [ ]:
for i in ['grade','employmentTitle', 'postCode', 'title','subGrade','term','employmentLength','earliesCreditLine']:
    print(i)
    print(df[i].unique())
    print('\n')

In [ ]:
##  对grade 进行编码
grade_dict={'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6}
df['grade']=df['grade'].map(grade_dict)

## 对subgrade 进行编码

subgrade_list=[]
for i in ['A','B','C','D','E','F','G']:
    for j in list(range(1,6)):
        subgrade_list.append(i+str(j))
        
t1=[i for i in range(35)]

subGrade_dict=dict(zip(subgrade_list,t1))
df['subGrade']=df['subGrade'].map(subGrade_dict)



#对term 进行编码

term_dict={3:0,5:1}
df['term']=df['term'].map(term_dict)


# 对employmentLength 进行编码
df['employmentLength'].unique()

#观察后发现好像不需要对其进行编码  其本来就是个连续的浮点数值

# 对employmenttitle 先进行分类  频数前100项作为独立变量  其余则分类为其他  ， 完成后再进行one-hot

In [ ]:
# 取前100的代号

top_100=list(df['employmentTitle'].value_counts().iloc[0:100,].index)


employmentTitle=[]
for i in range(len(df)):
    if df['employmentTitle'][i] not in top_100:
        employmentTitle.append('else')
    else:
        employmentTitle.append(df['employmentTitle'][i])
        

        
df['employmentTitle']=employmentTitle

In [ ]:
#引入一下sklearn 的label-encode包   主要是对ficoRangeLow / ficoRangeHigh 进行编码  对连续变量编码会方便一点

for col in tqdm(['ficoRangeLow','ficoRangeHigh']):
    le = LabelEncoder()
    le.fit(list(df[col].astype(str).values))
    df[col] = le.transform(list(df[col].astype(str).values))
    print('Label Encoding 完成')

In [ ]:
### 观察一下需要进行one-hot的变量  并添加到一个list中
dummy_list=['employmentTitle','homeOwnership','verificationStatus','purpose','regionCode','initialListStatus','applicationType']

In [ ]:
df=df.drop(columns=['policyCode'])
#前面已经提及policyCode只有一个值，没有研究意义，这里直接删去。

df=df.drop(columns=['issueDate'])
#因为懒 不想处理时间数据 因而删了

df=df.drop(columns=['postCode'])
#感觉邮政编码没什么用 留着一个地区编码就算了

df=df.drop(columns=['title'])
#title也太多了感觉没什么用

df=df.drop(columns=['id'])

df=df.drop(columns=['earliesCreditLine'])

df=df.drop(columns=['issueDateDT'])

# 开始进行one-hot处理

In [ ]:
for i in dummy_list:
    df=pd.concat([df,pd.get_dummies(df[i],prefix=i,prefix_sep='_')],axis=1)

In [ ]:
df=df.drop(columns=dummy_list)

In [ ]:
df

In [ ]:
df.to_csv('data_clean3.csv')

In [ ]:
df=df.dropna()
df=df.reset_index(drop=True)

data = df.drop(columns='isDefault')
target=df['isDefault']

data=data.drop(columns=['annualIncome/employmentLength', 'annualIncome*employmentLength'])
